# Lab: Ordered Hashtable

## Overview

For this assignment you will update and complete the implementation of the hashtable data structure presented in class, which exposes an API mirroring that of the built-in Python `dict`. When iterating over its contents (supported by the `__iter__`, `keys`, `values`, and `items` methods), your updated implementation will also reflect the order in which key/value pairs were originally inserted into the hashtable. This will require that you implement the two-tiered list system described during lecture.

The operations you will implement are listed alongside their descriptions below (`h` refers to a hashtable):

| Operation | Description |
|-----------|-------------|
| `h[k]`&nbsp;`=`&nbsp;`v` | If `h` does not contain key `k`, a new `k`&rightarrow;`v` mapping is added, else the value for key `k` is updated to `v`. |
| `h[k]`    | If `h` contains key `k`, the corresponding value is returned, else a `KeyError` is raised. |
| `del`&nbsp;`h[k]` | If `h` contains key `k`, it is removed along with its value, else a `KeyError` is raised. Note that if `k` is re-inserted at some later point it is considered a new key (for ordering purposes). |
| `k`&nbsp;`in`&nbsp;`h` | Returns `True` if key `k` is in `h`. |
| `len(h)` | Returns the number of keys in `h`. |
| `iter(h)` | Returns an iterator over all the keys in `h`, in the order they were added. |
| `h.keys()` | (Same as above) |
| `h.values()` | Returns an iterator over all the values in `h`, in the order they were added. |
| `h.items()` | Returns an iterator over all the key/value pairs (as tuples) in `h`, in the order they were added. |

Your hashtable will be provided with the initial number of buckets on creation (i.e., in `__init__`); your implementation must heed this value, as there may be performance ramifications if it does not.

In [38]:
class OrderedHashtable:
    class Node:
        """This class is used to create nodes in the singly linked "chains" in
        each hashtable bucket."""
        def __init__(self, index, next=None):
            # don't rename the following attributes!
            self.index = index
            self.next = next
        
    def __init__(self, n_buckets=1000):
        # the following two variables should be used to implement the "two-tiered" 
        # ordered hashtable described in class -- don't rename them!
        self.indices = [None] * n_buckets
        self.entries = []
        self.count = 0
        
    def __getitem__(self, key):
        idx = hash(key) % len(self.indices)
        n = self.indices[idx]

        while n:
            if self.entries[n.index]:
                if self.entries[n.index][0] == key:
                    return self.entries[n.index][1]
            n = n.next

        raise KeyError
        
    
    def __setitem__(self, key, val):
        idx = hash(key) % len(self.indices)
        n = self.indices[idx]
        if n:   # the n bucket already filled with a linkedlist
            while n.next:
                if self.entries[n.index]:
                    if self.entries[n.index][0] == key:
                        self.entries[n.index] = (key,val)
                        return
                n = n.next

            if n.index < len(self.entries):
                #print(n.index)
                #print(len(self.entries))
                if self.entries[n.index]:
                        if self.entries[n.index][0] == key:
                            self.entries[n.index] = (key,val)
                            return
            
            self.entries.append((key,val))
            self.count += 1
            
            if len(self.entries) > 0:
                n.next = OrderedHashtable.Node(len(self.entries)-1) 
            else:
                n.next = OrderedHashtable.Node(0) 
        
        else:   # nothing in the n bucket
            self.entries.append((key,val))
            self.count += 1
            if len(self.entries) > 0:
                self.indices[idx] = OrderedHashtable.Node(len(self.entries)-1) 
            else:
                self.indices[idx] = OrderedHashtable.Node(0) 

    def __delitem__(self, key):
        idx = hash(key) % len(self.indices)
        n = self.indices[idx]
        while n:
            if self.entries[n.index]:
                if self.entries[n.index][0] == key:
                    self.entries[n.index] = None
                    self.count -= 1
                    return
            n = n.next

        raise KeyError


    def __contains__(self, key):
        try:
            _ = self[key]
            return True
        except:
            return False
        
    def __len__(self):
        return self.count
    
    def __iter__(self):
        for i in self.entries:
            if i:
                yield i[0]
        
    def keys(self):
        return iter(self)
    
    def values(self):
        for i in self.entries:
            if i:
                yield i[1]
                
    def items(self):
        for i in self.entries:
            if i:
                yield i
                
    def __str__(self):
        return '{ ' + ', '.join(str(k) + ': ' + str(v) for k, v in self.items()) + ' }'
            
    def __repr__(self):
        return str(self)

In [18]:
# (3 tests) Short tests

from unittest import TestCase
import random

tc = TestCase()

ht = OrderedHashtable(2)

for k, v in (('batman', 'bruce wayne'), ('superman', 'clark kent'), ('spiderman', 'peter parker')):
    ht[k] = v
    
tc.assertEqual(len(ht), 3)

tc.assertEqual(ht['superman'], 'clark kent')

tc.assertTrue('spiderman' in ht)
tc.assertFalse('iron man' in ht)

with tc.assertRaises(KeyError):
    ht['iron man']

1
2


In [19]:
# (3 points) Basic tests (insertion, fetch, count, chain-lengths)

from unittest import TestCase
import random

tc = TestCase()

class MyInt(int):
    def __hash__(self):
        """MyInts hash to themselves — already current Python default, 
        but just to ensure consistency."""
        return self
    
def ll_len(l):
    """Returns the length of a linked list with head `l` (assuming no sentinel)"""
    c = 0
    while l:
        c += 1
        l = l.next
    return c
    
ht = OrderedHashtable(10)
for i in range(25):
    ht[MyInt(i)] = i*2

tc.assertEqual(len(ht), 25)

for i in range(5):
    tc.assertEqual(ll_len(ht.indices[i]), 3)
    
for i in range(5, 10):
    tc.assertEqual(ll_len(ht.indices[i]), 2)

for i in range(25):
    tc.assertTrue(MyInt(i) in ht)
    tc.assertEqual(ht[MyInt(i)], i*2)

0
10
1
11
2
12
3
13
4
14
5
15
6
16
7
17
8
18
9
19
10
20
11
21
12
22
13
23
14
24


In [23]:
# (3 points) Update testing

from unittest import TestCase
import random

tc = TestCase()

ht = OrderedHashtable(100)
d = {}

for i in range(100):
    k, v = str(i), str(i*2)
    d[k] = v
    ht[k] = v
    
for j in range(0, 100, 2):
    k, v = str(i), str(i*3)
    d[k] = v
    ht[k] = v
    
for j in range(0, 100, 4):
    k, v = str(i), str(i*4)
    d[k] = v
    ht[k] = v
    
for i in range(100):
    tc.assertTrue(k in ht)
    tc.assertEqual(d[k], ht[k])

0
6
5
13
1
19
4
21
14
30
8
32
18
36
19
40
43
44
30
45
15
46
33
47
47
48
17
51
16
52
38
55
9
56
34
60
22
63
31
64
58
65
37
66
65
69
52
70
63
72
40
75
55
76
69
80
74
82
39
83
35
84
59
85
42
86
60
87
84
90
51
94
56
95
80
96
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100
99
100


In [35]:
# (3 points) Deletion testing

from unittest import TestCase
import random

tc = TestCase()

ht = OrderedHashtable(100)
d = {}

for i in range(100):
    k, v = str(i), str(random.randrange(10000000, 99999999))
    d[k] = v
    ht[k] = v

for _ in range(50):
    k = str(random.randrange(100))
    if k in d:
        del d[k]
        del ht[k]

tc.assertEqual(len(ht), len(d))

for k,v in ht.items():
    tc.assertEqual(d[k], v)

0
6
5
13
1
19
4
21
14
30
8
32
18
36
19
40
43
44
30
45
15
46
33
47
47
48
17
51
16
52
38
55
9
56
34
60
22
63
31
64
58
65
37
66
65
69
52
70
63
72
40
75
55
76
69
80
74
82
39
83
35
84
59
85
42
86
60
87
84
90
51
94
56
95
80
96


In [32]:
# (4 points) Iteration order testing

from unittest import TestCase
import random

tc = TestCase()

ht = OrderedHashtable(1000)
l = [str(i) for i in range(0, 1000)]
random.shuffle(l)

for x in l:
    ht[x] = x

for _ in range(50):
    idx_to_del = random.randrange(len(l))
    val_to_del = l[idx_to_del]
    del ht[val_to_del]
    del l[idx_to_del]
    if random.randrange(2) == 0:
        l.append(val_to_del)
        ht[val_to_del] = val_to_del

for x, y in zip(l, ht):
    tc.assertEqual(x, y)

14
20
9
30
34
46
36
68
7
74
10
97
88
98
74
99
71
102
91
111
16
116
112
127
54
132
55
146
60
148
83
160
107
167
135
172
155
176
0
187
141
189
31
191
171
195
15
203
200
211
26
212
170
213
193
222
213
225
101
226
118
227
228
231
35
235
131
242
137
244
169
246
164
251
212
252
68
258
257
259
105
271
77
273
189
285
86
311
4
312
5
315
281
319
280
322
93
331
11
337
132
338
306
340
100
341
65
347
313
353
350
358
148
359
140
361
268
363
315
365
194
369
279
376
214
378
361
379
325
387
99
388
142
392
144
393
284
396
80
397
369
398
185
406
151
407
296
408
261
411
12
412
163
414
90
417
195
419
38
423
287
430
398
431
247
432
362
434
342
437
318
438
355
439
61
443
104
447
153
451
409
454
173
455
66
456
293
457
360
458
109
459
48
466
412
468
301
469
172
471
387
475
286
477
272
482
238
484
78
489
408
493
443
496
427
499
30
500
353
503
386
505
376
506
459
508
425
513
358
514
469
515
186
516
175
519
492
520
152
523
493
530
468
532
136
533
382
534
192
541
324
545
98
548
532
549
25
551
329
558
129
565
89
56

In [37]:
# (4 points) Stress testing

from unittest import TestCase
from time import time
import random

tc = TestCase()

ht = OrderedHashtable(100000)
d = {}

start = time()

for _ in range(100000):
    k, v = str(random.randrange(100000)), str(random.randrange(10000000, 99999999))
    d[k] = v
    ht[k] = v
    
for k,v in d.items():
    tc.assertTrue(k in ht)
    tc.assertEqual(d[k], ht[k])
    
end = time()
print(end-start)
tc.assertLess(end-start, 1.5, 'Your implementation ran too slow!')

0.5321445465087891
